In [5]:
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import os
import folium
import folium.plugins as plugins
from folium.features import DivIcon
from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units


%store -r file_name

met_data = pd.read_csv(file_name)
station_data = pd.read_csv('station_info.csv').rename(columns={'id':'station'})

merged_data = pd.merge(met_data, station_data, on='station')
# calculate the dewpoint and add the dataframe
td = dewpoint_from_relative_humidity(merged_data['tl'].values * units.degC, merged_data['rf'].values * units.percent)

merged_data['td'] = td.magnitude.round(decimals=1)

# extract = merged_data[merged_data['name'].str.contains(r'Mariazell|Lilienfeld|St.P')][['tl','time','name', 'lat', 'lon']]
extract = merged_data[['tl','tlmax','td','dd', 'time','name', 'lat', 'lon']]


# # Plot the responses for different events and regions
# sns.lineplot(x="time", y="tl",
#              hue="name",
#              data=extract)

# sns.despine()



In [6]:
extract.loc[extract['time'] == extract['time'].iloc[-1]]







,tl,tlmax,td,dd,time,name,lat,lon
51381,21.2,21.5,15.7,39.0,2024-08-27T16:30+00:00,Aflenz,47.54594,15.24069
51382,23.0,23.1,18.1,10.0,2024-08-27T16:30+00:00,Aigen im Ennstal,47.53278,14.13826
51383,22.8,22.9,15.9,14.0,2024-08-27T16:30+00:00,Allentsteig,48.69083,15.36694
51384,25.9,26.0,17.6,92.0,2024-08-27T16:30+00:00,Amstetten,48.10889,14.89500
51385,21.8,22.2,16.3,254.0,2024-08-27T16:30+00:00,Bad Aussee,47.61050,13.75844
...,...,...,...,...,...,...,...,...
51895,NaN,NaN,NaN,NaN,2024-08-27T16:30+00:00,Bad Radkersburg,46.68667,15.97861
51896,NaN,NaN,NaN,NaN,2024-08-27T16:30+00:00,Bad Radkersburg,46.68583,15.97944
51897,28.1,28.3,17.3,99.0,2024-08-27T16:30+00:00,Bad Radkersburg,46.69222,15.99333
51898,17.0,17.3,14.5,219.0,2024-08-27T16:30+00:00,Loibl Tunnel,46.44389,14.24917


In [8]:
# noe_values = extract.loc[extract['time'] == extract['time'][-1]]
latest = extract.loc[extract['time'] == extract['time'].iloc[-1]]

# iterate through each row and select
# 'Name' and 'Stream' column respectively.

for ind in latest.index:
    # print('Lat: {}, Lon: {}'.format(noe_values['lat'][ind], noe_values['lon'][ind]))
    lats = [latest['lat'][ind] for ind in latest.index]
    lons = [latest['lon'][ind] for ind in latest.index]
    temp = [latest['tl'][ind] for ind in latest.index]
    tlmax = [latest['tlmax'][ind] for ind in latest.index]
    tdew = [latest['td'][ind] for ind in latest.index]
    dd = [latest['dd'][ind] for ind in latest.index]
    name = [latest['name'][ind] for ind in latest.index]

type(temp[0])

numpy.float64

In [12]:


m = folium.Map(location=[lats[0], lons[0]], zoom_start=7, tiles=None)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=MglwGpuT1VUaVc2LSRX042X3krQz6y2cOZfGtNqWg3VMUke6gemUIWz7G1uxPPXP',
                 attr='CartoDB.Voyager', max_zoom=20).add_to(base_map)
base_map.add_to(m)

COLORS = [
    (lambda x: x > 40, "#EC8989"),
    (lambda x: 35 < x <= 40, "#EA3D09"),
    (lambda x: 30 < x <= 35, "#EA3D09"),
    (lambda x: 25 < x <= 30, "#EA6509"),
    (lambda x: 20 < x <= 25, "#C5C71B"),
    (lambda x: 15 < x <= 20, "#8EF700"),
    (lambda x: 10 < x <= 15, "#07AF02"),
    (lambda x: 5 < x <= 10, "#94EA7F"),
    (lambda x: 0 < x <= 5, "#9cecc8"),
    (lambda x: 0 >= x >= -5, "#18B5DA"),
    (lambda x: -5 > x >= -10, "#1661e8"),
    (lambda x: -10 > x >= -15, "#A375E6"),
    (lambda x: -15 > x >= -20, "#ce8bf0"),
    (lambda x: x < -20, "#EC9CE6"),
]



# This picks the color for temperature and tmax (potentially also dewpoint)
def get_color(x):
    for predicate, color in COLORS:
        if predicate(x):
            return color

    return 'black'

# Add analysis for temperature
fg = folium.FeatureGroup(name="Temperature", control=True, overlay=False).add_to(m)

for i in range(len(lats)):
        color = get_color(temp[i])
        html_str = '<div style="font-size: 18pt; color : ' + color +'">' + str(temp[i]) + '</div>'
        p1 = [lats[i], lons[i]]
        folium.Marker(p1, icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fg)
 
 
# Add analysis for maximum temperature        
fgg = folium.FeatureGroup(name="Tmax", control=True, overlay=False, show=False).add_to(m)

for i in range(len(lats)):
        color = get_color(tlmax[i])
        html_str = '<div style="font-size: 18pt; color : ' + color +'">' + str(tlmax[i]) + '</div>'
        p1 = [lats[i], lons[i]]
        folium.Marker(p1, icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgg)
        


# Add analysis for dewpoint temperature        
fggg = folium.FeatureGroup(name="Dewpoint", control=True, overlay=False, show=False).add_to(m)

for i in range(len(lats)):
        color = get_color(tdew[i])
        html_str = '<div style="font-size: 18pt; color : ' + color +'">' + str(tdew[i]) + '</div>'
        p1 = [lats[i], lons[i]]
        folium.Marker(p1, icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fggg)
        



# Add analysis for wind direction of mean 10 min speed
ddg = folium.FeatureGroup(name="Wind direction", control=True, overlay=False, show=False).add_to(m)

for i in range(len(lats)):
    rotate_str = str(dd[i] + 90)
    folium.Marker(location=[lats[i], lons[i]], popup=name[i],
                            icon=plugins.BeautifyIcon(icon="fa-solid fa-arrow-right",
                                            border_color='transparent',
                                            background_color='transparent',
                                            border_width=1,
                                            text_color='black',
                                            inner_icon_style='margin:0px;font-size:4em;transform: rotate({0}deg);'.format(rotate_str))).add_to(ddg)

folium.LayerControl().add_to(m)

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


# icon = folium.DivIcon(html="17.4")
# folium.Marker([47.0, 15.0], icon=icon).add_to(m)
m.save(os.path.join('simple_popups.html'))